In [5]:
import pandas as pd
import mygene

In [16]:
flat_file_url = "https://gist.github.com/kdaily/2ed85e0dd3048fea8424b40243ddfa1c/raw/420086bd941962df66992667972c13462e504cc6/gencode.v24.primary_assembly.refFlat.txt"
gene_table = pd.read_csv(flat_file_url, sep='\t', header=None, usecols=[0], names=['ensembl_gene_id'])
gene_table = gene_table["ensembl_gene_id"].replace("\\..*", "", regex=True).drop_duplicates().reset_index()

mg = mygene.MyGeneInfo()
bioconductor_gene_info = mg.getgenes(gene_table['ensembl_gene_id'], fields=["symbol", "name", "summary", "type_of_gene", "alias"], as_dataframe=True)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-33000...done.
querying 33001-34000...done.
querying 34001-35000...done.
queryin

In [20]:
bioconductor_gene_info.index.rename("ensembl_gene_id", inplace=True)

_id             56712
_version        56712
name            40716
symbol          40716
type_of_gene    24835
alias           19242
summary         14310
notfound         4015
dtype: int64


In [48]:
duplicates = bioconductor_gene_info.groupby('symbol', as_index=False).agg('count')
print(duplicates[duplicates['_id'] > 1]) # effectively the same list we found on the ticket

          symbol  _id  _version  name  type_of_gene  alias  summary  notfound
0        5S_rRNA    4         4     4             0      0        0         0
1      5_8S_rRNA    6         6     6             0      0        0         0
2            7SK    7         7     7             0      0        0         0
322     ACTR3BP2    2         2     2             1      1        0         0
836      ALG1L9P    2         2     2             1      0        0         0
...          ...  ...       ...   ...           ...    ...      ...       ...
37210         U6   13        13    13             0      0        0         0
37211         U7    7         7     7             0      0        0         0
37212         U8   22        22    22             0      0        0         0
38065      Vault    4         4     4             0      0        0         0
38480      Y_RNA  756       756   756             0      0        0         0

[93 rows x 8 columns]


In [49]:
print(duplicates['ensembl_gene_id'])

             symbol  _id  _version  name  type_of_gene  alias  summary  \
0           5S_rRNA    4         4     4             0      0        0   
1         5_8S_rRNA    6         6     6             0      0        0   
2               7SK    7         7     7             0      0        0   
3              A1BG    1         1     1             1      1        1   
4          A1BG-AS1    1         1     1             1      1        0   
...             ...  ...       ...   ...           ...    ...      ...   
39522         ZZEF1    1         1     1             1      1        0   
39523          ZZZ3    1         1     1             1      1        0   
39524  hsa-mir-1253    1         1     1             0      0        0   
39525   hsa-mir-423    1         1     1             0      0        0   
39526       snoZ196    1         1     1             0      0        0   

       notfound  
0             0  
1             0  
2             0  
3             0  
4             0  
...

In [50]:
missing_symbol = bioconductor_gene_info[bioconductor_gene_info['symbol'].isna()]
missing_symbol

,_id,_version,name,symbol,type_of_gene,alias,summary,notfound
ensembl_gene_id,,,,,,,,
ENSG00000274890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
ENSG00000238009,ENSG00000238009,1.0,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000239945,ENSG00000239945,1.0,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000268903,ENSG00000268903,1.0,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000269981,ENSG00000269981,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
ENSG00000274938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
ENSG00000278806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
ENSG00000274152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [58]:
import synapseclient
syn = synapseclient.Synapse()
syn.login()

median_expression = 'syn12514804' # csv
rna_seq_differential_expression = 'syn14237651' # tsv
proteomics = 'syn18689335' # csv
metabolomics = 'syn26064497' # feather

median_expression = syn.get(median_expression)
rna_seq_differential_expression = syn.get(rna_seq_differential_expression)
proteomics = syn.get(proteomics)
metabolomics = syn.get(metabolomics)

median_expression = pd.read_csv(median_expression.path)
rna_seq_differential_expression = pd.read_csv(rna_seq_differential_expression.path, sep='\t')
proteomics = pd.read_csv(proteomics.path)
metabolomics = pd.read_feather(metabolomics.path)

Welcome, Matthew Fazza!

(114988, 3)
(1330018, 18)
(22704, 10)
(15401, 10)


Finding the name of the ensembl_gene_id column:

In [64]:
for dataset in [median_expression, rna_seq_differential_expression, proteomics, metabolomics]:
    [print(col) for col in dataset.columns if 'ens' in col.lower()]

ensembl_gene_id
ensembl_gene_id
ENSG
ensembl_gene_id


Setting aside the missing ensembl_gene_ids:

In [76]:
missing_ids = pd.Series(missing_symbol.index)
missing_ids.count()

20011

Available data from missing data.

In [88]:
for df in [median_expression, rna_seq_differential_expression, proteomics, metabolomics]:
    try:
        print(df[df['ensembl_gene_id'].isin(missing_ids)].dropna().shape[0])
        print()
    except:
        print(df[df['ENSG'].isin(missing_ids)].dropna().shape[0])
        print()

7386

3282

23

4842

